<a href="https://colab.research.google.com/github/Malerion/Archives-Python/blob/main/NLP_5_TFLDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive


/content/drive/MyDrive


In [3]:
import zipfile

with zipfile.ZipFile('tweets.zip', 'r') as zip_ref:
    zip_ref.extractall('destination_folder')


In [4]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/destination_folder/train.csv')
df.head()


,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [5]:
#exercice 2
# Exclure les tweets neutres
df_filtered = df[df['sentiment'].isin(['positive', 'negative'])]

# Calculer le pourcentage de tweets positifs/négatifs
positive_percentage = (df_filtered['sentiment'] == 'positive').mean() * 100
negative_percentage = (df_filtered['sentiment'] == 'negative').mean() * 100

print(f"Pourcentage de tweets positifs : {positive_percentage:.2f}%")
print(f"Pourcentage de tweets négatifs : {negative_percentage:.2f}%")


Pourcentage de tweets positifs : 52.45%
Pourcentage de tweets négatifs : 47.55%


In [10]:
#exo 3
from sklearn.model_selection import train_test_split

# Copie des colonnes dans des Séries
X = df[df['sentiment'].isin(['positive', 'negative'])]['text']
y = df[df['sentiment'].isin(['positive', 'negative'])]['sentiment']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=32)


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Création du modèle vectorizer et entraînement sur X_train
tfidf_vectorizer = TfidfVectorizer()
X_train_CV_tfidf = tfidf_vectorizer.fit_transform(X_train)

X_test_CV_tfidf = tfidf_vectorizer.transform(X_test)

# Vérification des dimensions
print("Dimensions de X_test_CV_tfidf :", X_test_CV_tfidf.shape)

# Nombre d'éléments stockés dans la matrice
print("Nombre d'éléments stockés dans la matrice (stored elements) :", X_test_CV_tfidf.nnz)


Dimensions de X_test_CV_tfidf : (4091, 15806)
Nombre d'éléments stockés dans la matrice creuse (stored elements) : 44633


In [13]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings

# Ignorer les avertissements de convergence
warnings.filterwarnings("ignore", category=UserWarning)

# Initialisation et entraîneùmant du modèle de régression logistique
logistic_regression_tfidf = LogisticRegression(max_iter=1000)
logistic_regression_tfidf.fit(X_train_CV_tfidf, y_train)

# Prédictions ds données d'entraînement et de test
y_train_pred_tfidf = logistic_regression_tfidf.predict(X_train_CV_tfidf)
y_test_pred_tfidf = logistic_regression_tfidf.predict(X_test_CV_tfidf)

# Précision des données d'entraînement
accuracy_train_tfidf = accuracy_score(y_train, y_train_pred_tfidf)
print("Précision sur les données d'entraînement :", round(accuracy_train_tfidf, 3))

# Précision sur les données de test
accuracy_test_tfidf = accuracy_score(y_test, y_test_pred_tfidf)
print("Précision sur les données de test :", round(accuracy_test_tfidf, 3))


Précision sur les données d'entraînement : 0.932
Précision sur les données de test : 0.873
